Load the environment variables from a .env file

To get an api key, navigate to https://aistudio.google.com/apikey to create one. Then create a .env file in the same directory as this notebook with the following content:
```GOOGLE_API_KEY=your_api_key_here```


In [25]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

Single shot question answering with Gemini 2.0 Flash to check correct working of the code

In [26]:
from google import genai
from pydantic import BaseModel

class Recipe(BaseModel):
    medicijn: str
    categorie: str
    recept: str
    medische_vraag: str

client = genai.Client(api_key=api_key)
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Ik heb een infectie waarvoor ik een antibioticum nodig heb. Welk middel kan ik gebruiken?",
    config={
        "response_mime_type": "application/json",
        "response_schema": Recipe,
        "system_instruction": "Formuleer het recept volgens template 'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. Geef bij de instructie ook de duur van de behandeling aan.",
    },
)

print(response.text)

{
  "medicijn": "Amoxicilline",
  "categorie": "Antibioticum",
  "recept": "Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. driemaal daags 1 capsule gedurende 7 dagen.",
  "medische_vraag": "Ik heb een infectie waarvoor ik een antibioticum nodig heb. Welk middel kan ik gebruiken?"
}


Create a function to get a recipe based on a medical question

In [27]:
def get_recipe(medical_question: str) -> Recipe:
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=medical_question,
        config={
            "response_mime_type": "application/json",
            "response_schema": Recipe,
            "system_instruction": "Formuleer het recept volgens template 'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. Geef bij de instructie ook de duur van de behandeling aan.",
        },
    )
    return response.parsed

recipe = get_recipe("Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?")
print(recipe.model_dump_json())


{"medicijn":"Omeprazol","categorie":"Maagbeschermer","recept":"Rp. Omeprazol 20mg capsule\ndtd. No. 30\nS. 1 capsule per dag gedurende 4 weken.","medische_vraag":"Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?"}


In [44]:
import pandas as pd
filepath = "../data/unieke_vragen.json"
questions = pd.read_json(filepath)

questions

,0
0,Ik heb een keelontsteking. Heb ik antibiotica ...
1,Ik heb blaasontstekingklachten. Wat kan ik daa...
2,Ik heb een longontsteking. Welke antibiotica z...
3,Ik heb wondinfectieverschijnselen. Welk antibi...
4,Ik heb een tandvleesontsteking. Kan ik hier ie...
...,...
194,Ik heb slikproblemen. Zijn er medicijnen om de...
195,Ik heb spastische spieren door MS. Wat kan ik ...
196,Ik heb MS. Wat zijn mijn onderhoudsmedicijnen?
197,Ik heb een vergrote schildklier. Moet ik iets ...


In [48]:
import json

recipes = []

for question in questions.head(10)[0].tolist():
    try:
        recipe = get_recipe(question)
        recipes.append(recipe)
        print(f"Recipe generated: {recipe.model_dump()}")
    except Exception as e:
        print(f"Error generating recipe for question '{question}': {e}")

# Write the recipes to a JSON file
output_file = "../data/generated_recipes.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump([recipe.model_dump() for recipe in recipes], f, ensure_ascii=False, indent=4)


Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibioticum', 'recept': 'Rp. Amoxicilline 500mg capsule\ndtd. No. 21\nS. driemaal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Keelontsteking, mogelijk antibiotica nodig'}
Recipe generated: {'medicijn': 'Nitrofurantoine', 'categorie': 'Blaasontsteking', 'recept': 'Rp. Nitrofurantoine 100mg capsule\ndtd. No. 20\nS. 3 maal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Ik heb blaasontstekingklachten. Wat kan ik daartegen nemen?'}
Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibiotica', 'recept': 'Rp. Amoxicilline 500mg tabletten\ndtd. No. 21\nS. 3 maal daags 1 tablet gedurende 7 dagen.', 'medische_vraag': 'Welke antibiotica zijn geschikt bij een longontsteking?'}
Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibioticum', 'recept': 'Rp. Amoxicilline 500mg capsule\ndtd. No. 21\nS. Driemaal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Wondinfectieverschijnselen'}
R

Check each recipe for correctness. Correctness is defined as follows:
- The recipe should contain a medicine name, strength, form, number of doses, and instructions.
- The medicine name should be a valid medicine name.
- The strength should be a valid strength.
- The form should be a valid form (e.g., tablet, capsule, liquid).
- The number of doses should be a valid number.
- The instructions should include the duration of the treatment.